In [ ]:
!pip install -U -q PyDrive
!pip install -q tflite-model-maker

In [ ]:
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader
from tflite_model_maker.config import ExportFormat

import tensorflow as tf
assert tf.__version__.startswith('2')

import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

In [ ]:
# descagar los archivos :D
file_list = drive.ListFile(
    {'q': "'1GZ-MdkCztGtWEUhj73MPV3p74V3u08As' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

Descromprimir
!unzip /root/data/FlagsDataset.zip -d /root/data/FlagsDataset

In [ ]:
data = DataLoader.from_folder('/root/data/lagsDataset')

In [ ]:
train_data, test_data = data.split(0.9)
model = image_classifier.create(train_data, validation_data=test_data, epochs=50, learning_rate = 0.001, batch_size=16)
loss, accuracy = model.evaluate(test_data)

In [ ]:
# Guardar el Modelo
model.export(export_dir='/root/data/', export_format=ExportFormat.SAVED_MODEL)

In [ ]:
#convertit modelo a float
converter = tf.lite.TFLiteConverter.from_saved_model('/root/data/saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_unquant = converter.convert()

with open('/root/data/model_unquant.tflite', 'wb') as f:
  f.write(tflite_model_unquant)

In [ ]:
files.download('/root/data/model_unquant.tflite')